In [4]:
import pandas as pd

ValueError: source code string cannot contain null bytes

In [3]:
#importing Libraries

import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns
import plotly.offline as py
import plotly_express as px


from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

from IPython.display import Image
warnings.filterwarnings('ignore')
%matplotlib inline

ValueError: source code string cannot contain null bytes

# LOADING THE DATA

In [ ]:
df=pd.read_csv("covid19_india.csv")

In [ ]:
df_one=pd.read_csv("India.csv")

# DATA OF INDIA (COVID)

In [ ]:
df

In [ ]:
df_one

In [ ]:
df.tail(15)# recent  15 days's cases statewise

In [ ]:
df_one.tail(15)#recent  15 days's cases of Country

In [ ]:
df.dtypes

In [ ]:
df_one.dtypes

In [ ]:
df.info()

In [ ]:
df_one.info()

In [ ]:
df.isnull()

In [ ]:
df_one.isnull()

In [ ]:
df.isnull().sum() #checking for null values

In [ ]:
df_one.isnull().sum()

In [ ]:
state_cases = df.groupby('State')['Confirmed','Deceased','Recovered'].max().reset_index()

state_cases

In [ ]:
#Creating new columns 'Active', 'Death rate per 100' and 'Cure rate per 100'
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Recovered'] + state_cases['Deceased'])
state_cases

In [ ]:
#Sort by maximum Confirmed Cases
state_cases = state_cases.sort_values(by='Confirmed', ascending=False) 
state_cases.style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deceased"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Blues', subset=["Active"])\
                        

# VISUALIZATION OF COVID-19 INDIA CASES

In [ ]:
confirmed = state_cases['Confirmed'].sum()
recovered = state_cases['Recovered'].sum()
deaths = state_cases['Deceased'].sum()
active = state_cases['Active'].sum()

labels = ['Active','Recovered','Deceased']
sizes = [active,recovered,deaths]
color= ['#66b3ff','green','red']
explode = []

for i in labels:
    explode.append(0.05)
    
plt.figure(figsize= (15,10))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=9, explode =explode,colors = color)
centre_circle = plt.Circle((0,0),0.70,fc='white')

fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('INDIA COVID-19 Cases',fontsize = 20)
plt.axis('equal')  
plt.tight_layout()

# Plot for Total Confirmed cases in India (State wise Anlaysis)

Bar plot of Confirmed cases statewise

In [ ]:

import datetime 
today='28-10-2020'
#today = datetime.date.today() # today date
yesterday = '27-10-2020'
df1= df[df['Date']==yesterday] # Below the plot I have shown the dataframe
fig = px.bar(df1, x='State', y='Confirmed', height=600)
fig.update_layout(
    title='Till {} Total Confirmed Cases in India'.format(yesterday))
fig.show()
#df1

Plot for Total Death cases in India (State wise Anlaysis)

In [ ]:
#plot for Total Death cases in India (State wise Anlaysis)
import datetime 
today='28-10-2020'
#today = datetime.date.today() # today date
yesterday = '27-10-2020'
df1= df[df['Date']==yesterday] # Below the plot I have shown the dataframe
fig = px.bar(df1, x='State', y='Deceased', height=600)
fig.update_layout(
    title='Till {} Total Death Cases in India'.format(yesterday))
fig.show()
#df1

Plot for Total Active cases in India (State wise Anlaysis)

In [ ]:
#plot for Total Active cases in India (State wise Anlaysis)
import datetime 
df1= state_cases # Below the plot I have shown the dataframe
fig = px.bar(df1, x='State', y='Active', height=600)
fig.update_layout(
    title='Till {} Total Active Cases in India'.format(yesterday))
fig.show()
#df1

# DATE WISE DATA PLOTTING OF PREVIOUS MONTH

In [ ]:
region_dict = dict()
regions = df.groupby('State')
for region, group in regions:
    region_dict[region]=group



In [ ]:
 for name, obj in region_dict.items():
        plt.scatter(x=obj.Date, y=obj.Confirmed, s=50,label=name, alpha=0.8)
        plt.legend()
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.show()  


In [ ]:
 for name, obj in region_dict.items():
        plt.scatter(x=obj.Date, y=obj.Deceased, s=50,label=name, alpha=0.8)
        plt.legend()
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.show()  


In [ ]:
 for name, obj in region_dict.items():
        plt.scatter(x=obj.Date, y=obj.Recovered, s=50,label=name, alpha=0.8)
        plt.legend()
        plt.xticks(rotation=90)
        plt.grid(True)
        plt.show()  


In [ ]:
import datetime
import matplotlib.dates as mdates

In [ ]:
sns.lineplot(x="Date", y="Confirmed",legend = 'full' , data=df_one)

In [ ]:
categories = df.State.unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
l=LabelEncoder()
label = l.fit_transform(df["State"])

In [ ]:
df["State_num"]=label

In [ ]:
categories_num = df.State_num.unique()
categories_num=np.sort(categories_num)
categories_num

In [ ]:
categories=np.sort(categories)
categories

In [ ]:
res = dict(zip(categories,categories_num))
res

In [ ]:
df["map_State"] = df["State"].map(res)
df

In [ ]:
df["Date"]=pd.to_datetime(df["Date"])
df["Date"] = df["Date"].dt.strftime('%Y%m%d')
df['Date'] = df['Date'].astype(int)

In [ ]:
X=df[['Date','map_State']]
y=df[['Confirmed']]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def bestparam_randCV(model,hyperparam,x_train, y_train, n_iter=1000):
    
    hyperparam = hyperparam
    randomizedCV = RandomizedSearchCV(model, param_distributions = hyperparam, cv = 10,
                                          n_iter = n_iter, scoring = 'neg_mean_squared_error', n_jobs=-1, 
                                          random_state = 42, verbose = True)
    randomizedCV.fit(x_train, y_train)
    

    print ('Best MSE', randomizedCV.score(x_train, y_train))
    print ('Best Param', randomizedCV.best_params_)
    return randomizedCV

In [ ]:

reg         = DecisionTreeRegressor()             

hyperparam =  {'max_features': ['auto', 'sqrt'],
               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10]}
 
n_iter     = 100
best_dt1   = bestparam_randCV(reg, hyperparam, X_train, y_train, n_iter)

In [ ]:
dt1 = DecisionTreeRegressor(
                        max_features      = best_dt1.best_params_.get('max_features'),
                        max_depth         = best_dt1.best_params_.get('max_depth'),
                        min_samples_split = best_dt1.best_params_.get('min_samples_split'),
                        min_samples_leaf  = best_dt1.best_params_.get('min_samples_leaf'))

result_dt1 = dt1.fit(X_train, y_train)

In [ ]:
data_test = X_test.copy()
data_test['Confirmed_test_score']=y_test['Confirmed']

In [ ]:
data_test['Confirmed_test_score_pred_DTR_HT'] = result_dt1.predict(X_test).astype(int)

In [ ]:
data_test

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
AA=r2_score(y_test['Confirmed'],data_test['Confirmed_test_score_pred_DTR_HT'])
print("ACCURACY OF MODEL FOR PREDICTION OF TEST DATA USING DST HYPER TUNING USING RANDOMIZED SEARCH:{}".format(AA))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model_rf = RandomForestRegressor(n_estimators=5000, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
a=r2_score(y_train, pred_train_rf)
print(a)
data_test['RFR_PREDICTION'] = model_rf.predict(X_test).astype('int64')
ACCURACY=r2_score(y_test['Confirmed'],data_test['RFR_PREDICTION'])
print("ACCURACY OF MODEL FOR PREDICTION OF TEST DATA USING RANDOM FOREST REGRESSOR:{}".format(ACCURACY))

In [ ]:
data_test

In [ ]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train,y_train)

data_test['rfr_forecast'] = rf.predict(X_test)
ACCURACY=r2_score(y_test['Confirmed'],data_test['rfr_forecast'])
print("ACCURACY OF MODEL FOR PREDICTION OF TEST DATA USING RANDOM FOREST REGRESSOR:{}".format(ACCURACY))

In [ ]:
data_test

In [ ]:
xgb = XGBRegressor(n_estimators=100)
xgb.fit(X_train,y_train)

data_test['xgb_forecast'] = xgb.predict(X_test)
ACCURACY=r2_score(y_test['Confirmed'],data_test['xgb_forecast'])
print("ACCURACY OF MODEL FOR PREDICTION OF TEST DATA USING XGBOOST REGRESSOR:{}".format(ACCURACY))

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train,y_train)
reg.score(X_train,y_train)